In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import collections
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)
import pandas as pd
import random
from deap import creator, tools, base, gp
from sklearn.metrics import confusion_matrix
import operator


# TODO: WRITE PREPROCESS FUNCTION THAT RETURNS X_test and Y_test
# Should also return solutions to survived / not
def preprocess():
    
    convAge = lambda x: int(x) if x else 0
    convSex = lambda x: bool(1) if str(x) == 'female' else bool(0)
    convEmbarked = lambda c: ord(c) if c else 0
    
    
    raw_train_data = pd.read_excel('train.xls', 'train', converters={'Survived': bool, 'Sex': convSex}) 
    processed_data = raw_train_data.drop(['PassengerId', 'Name', 'Ticket'], axis=1) 
    processed_data['Age'] = processed_data['Age'].fillna(int(processed_data['Age'].mean()))
    processed_data['Embarked'] = processed_data['Embarked'].fillna(processed_data['Embarked'].mode()[0])
    processed_data['EmbarkedC'] = processed_data['Embarked'] == 'C' 
    processed_data['EmbarkedS'] = processed_data['Embarked'] == 'S' 
    processed_data['EmbarkedQ'] = processed_data['Embarked'] == 'Q' 
    processed_data['Pclass1'] = processed_data['Pclass'] == 1 
    processed_data['Pclass2'] = processed_data['Pclass'] == 2 
    processed_data['Pclass3'] = processed_data['Pclass'] == 3 
    processed_data['Cabin'] = processed_data['Cabin'].apply(lambda x: x[0] if not pd.isnull(x) else x)
    processed_data['CabinA'] = processed_data['Cabin'] == 'A'
    processed_data['CabinB'] = processed_data['Cabin'] == 'B'
    processed_data['CabinC'] = processed_data['Cabin'] == 'C'
    processed_data['CabinD'] = processed_data['Cabin'] == 'D'
    processed_data['CabinE'] = processed_data['Cabin'] == 'E'
    processed_data['CabinF'] = processed_data['Cabin'] == 'F'
    processed_data['CabinG'] = processed_data['Cabin'] == 'G'
    processed_data['CabinNone'] = (processed_data['Cabin'] >= 'A') & (processed_data['Cabin'] <= 'G')
    #(processed_data['Cabin'] != A) & (processed_data['Cabin'] != B) & (processed_data['Cabin'] != C) & (processed_data['Cabin'] != D) & (processed_data['Cabin'] != E) & (processed_data['Cabin'] != F) & (processed_data['Cabin'] != G)

    survived_table = processed_data['Survived'].values
    processed_data = processed_data.drop(['Survived', 'Embarked', 'Pclass','Cabin'], axis=1)
    #print(processed_data)
    #print(survived_table)
        
    return (processed_data, survived_table)

processed_data, survived_table = preprocess()
# print('Processed data: \n', processed_data)
# print('survived table: \n', survived_table)

In [2]:
# CONSTANTS
NUM_GEN = 50  # Number of generation 
IN_POP = 200  # Initial Population Size
# CHILD_POP = 100 # Not sure if needed
SEED = 'Iceberg'     # Seed for the random object 
CX_PB = 0.5 # probability of mating / crossover
MUT_PB = 0.2 # probability of mutating
# HOF_MAX = 100 # number of hall of fame entries

In [3]:
# minimize both objectives (FP AND FN)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0)) 

# each individual will be a lisp tree of primitives with attribute FitnessMin
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin) 

In [4]:
random.seed(SEED) # seeds random

In [5]:
# CREATE PSET WITH ATTRIBUTES
# PREFERABLY MAKE STRONGLY TYPED SET
pset = gp.PrimitiveSetTyped("MAIN", [bool, bool, bool, bool, float, float, float, float, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool, bool], bool)
pset.addPrimitive(np.add, [float, float], float)
pset.addPrimitive(np.multiply, [float, float], float)
pset.addPrimitive(np.logical_and, [bool, bool], bool)
pset.addPrimitive(np.logical_or,[bool, bool], bool)
pset.addPrimitive(np.logical_not, [bool], bool)
pset.addPrimitive(np.less_equal, [float, float], bool)
pset.addPrimitive(np.greater_equal, [float, float], bool)
pset.addTerminal(0, float)
pset.addTerminal(1, float)
pset.addTerminal(0.5, float)
pset.addTerminal(-1, float)

pset.renameArguments(ARG0='Sex',ARG1='Age',ARG2='SibSP',ARG3='ParCh',ARG4='Fare',ARG5='EmbarkedC',ARG6='EmbarkedS',\
                     ARG7='EmbarkedQ',ARG8='Pclass1',ARG9='Pclass2',ARG10='Pclass3',Arg11='CabinA',Arg12='CabinB',
                     Arg13='CabinC',Arg14='CabinD',Arg15='CabinE',Arg16='CabinF',Arg17='CabinG',Arg18='CabinNone')

In [6]:
# CREATES TOOLBOX WITH INDIVIDUAL LISP TREES
toolbox = base.Toolbox()

# genHalfAndHalf generates a full tree 50% of the time and an incomplete one the other 50%
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=4)

# puts the random tree into the generated individual
# essentially a constructor for the individual
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)

# creates a list of the individuals
# essentially constructor for population
# generates a population of size n comprising of random individuals 
# calles as toolbox.population(n)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# converts the lisp tree into a function that can be evaluated
toolbox.register("compile", gp.compile, pset=pset)

In [7]:
# will evaluate individuals predictions against the solutions to the training data

# EVALUATION FUNCTION
# ONLY NEEDS TO TAKE IN AN INDIVIDUAL
# SHOULD MAKE X_TEST AND Y_TEST GLOBAL WHICH WILL BE USED TO EVALUATE
def evalTitanic(individual, pset):
    results = [0] * len(survived_table)
    func = gp.compile(expr=individual, pset=pset)
    for i, row in enumerate(processed_data.values):
        results[i] = func(*row)
        
    FP, FN = 0,0
    for a, b in zip(results, survived_table) :
        if bool(a) is True and bool(b) is False:
            FP += 1
        elif bool(a) is False and bool(b) is True:
            FN += 1

    return (FP, FN)

# CAN BE USED TO GENERATE NEW TEST INDIVIDUAL FOR DEBUGGING
# a_given_individual = toolbox.population(n=1)[0]
# a_given_individual.fitness.values = toolbox.evaluate(a_given_individual)
# a_given_individual.fitness.values

In [8]:
def areSimilar(ind1, ind2):
    return ind1.fitness.values[0] == ind2.fitness.values[0] and ind1.fitness.values[1] == ind2.fitness.values[1]

In [9]:
# REGISTER TOOLBOX FUNCTIONS
# FUNCTIONS THAT WILL BE CALLED DURING THE EVOLUTIONARY PROCESS

# EVALUATE FUNCTION

# ONLY NEEDS TO PASS IN THE INDIVIDUAL
toolbox.register("evaluate", evalTitanic, pset=pset)

# method to select individuals that will mate
# http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.selTournament
toolbox.register("select", tools.selSPEA2)

# mating method uses cxOnePoint
toolbox.register("mate", gp.cxOnePoint)

# creates a random full tree with min <= depth <= max
toolbox.register("expr_mut", gp.genFull, min_=1, max_=4)

# Randomly select a point in the tree individual, then replace the subtree at that point
# as a root by the expression generated using method expr().
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [10]:
# SETS MAX VALUES FOR DELIMITAION

# SETS MAXIMUM HEIGHT OF TREE AS A RESULT OF MATING TO max_value
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

# SETS MAXIMUM HEIGHT OF TREE AS A RESULT OF MUTATION TO max_value
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [11]:
# PARETO DOMINANCE FUNCTION
# NOT SURE IF REQUIRED AS Hall of Fame might take care of this

# takes in two indiviudals
# returns true if the first individual dominates the second individual.
def pareto_dominance(ind1, ind2):
    not_equal = False
    for value_1, value_2 in zip(ind1.fitness.values, ind2.fitness.values):
        if value_1 > value_2:
            return False
        elif value_1 < value_2:
            not_equal = True
    return not_equal

In [12]:
# keeps track of best individuals over 
# http://deap.readthedocs.io/en/master/api/tools.html#hall-of-fame
# http://deap.readthedocs.io/en/master/api/tools.html#deap.tools.ParetoFront
# hof = tools.HallOfFame(maxsize = HOF_MAX)  # might have to use tools.ParetoFront() instead
# hof = tools.ParetoFront(areSimilar)  
hof = tools.ParetoFront()  
# specifies how many individuals we want in the hall of fame
# maxsize keeps the best 100

In [13]:
gen = range(NUM_GEN)
# gen = range(20)
# avg_list = []
# max_list = []
# min_list = []
# PROBABLY JUST USE ABOVE FOR PRESENTATION PURPOSES

aucList = [] # holds the area under the curve of each generation

# CREATES POPULATION OF SIZE = IN_POP
pop = toolbox.population(n=IN_POP)

# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [14]:
# method which gets the best individuals from each population taking into account best individuals that have existed
# returns a list of size IN_POP
def getBestFromPopulation(gen_offspring):
    
    # creates a deep coopy of the generated offsping
    cloned_offspring = list(map(toolbox.clone, gen_offspring))
    
    # creates a population list comprising of best individuals that have ever existed and the current offspring
    total_pop = cloned_offspring + list(hof) 
    
    # will hold the list of the best individuals comprising of best individuals that have ever 
    # existed and the current offspring
    best_pop = []
    
    tempHOF = tools.ParetoFront(areSimilar) # will be used to keep track of best individuals of different rankings
    # tempHOF now holds non-dominated individuals
#     print('TOTAL_POP = LEN(CLONED) + LEN(HOF): ', (len(total_pop) == len(cloned_offspring) + len(hof)))
#     print('PASSED IN OFFSPRING')
#     for off in cloned_offspring:
#         print(off.fitness.values)
#     print('HOF')
#     for famer in tempHOF:
#         print(famer.fitness.values)
    while len(total_pop) != 0: # ends when total_pop is empty
#         print('INIT LENGTH: ', len(total_pop))
#         print('\n\nIN LOOP')
        tempHOF.update(total_pop) # populates tempHOF with the cumulative population list
        tempList = [famer for famer in tempHOF]
        np.random.shuffle(tempList)
        for famer in tempList: # iterates through all non-dominated individuals
            if len(best_pop) == IN_POP:
#                 print('RETURNING: ', len(best_pop))
#                 for best in best_pop:
#                     print(best.fitness.values)
                return best_pop # returns a list of size IN_POP
            best_pop += [famer] # adds famer to list of best population
#             print('adding: ', famer.fitness.values)
            total_pop = [individual for individual in total_pop if individual != famer]
            # removes all occurences of the famer from the list
#             print('ON REMOVING FAMER: ', len(total_pop))
            
        tempHOF.clear() # Empties temporary hall of fame
#         print('\nCLEARED?: ', len(tempHOF) == 0)
#     print('RETURNING: ', len(best_pop))
#     for best in best_pop:
#         print(best.fitness.values)
    return best_pop

In [15]:
def dispGraphwithAUC(pop):
    # collects fitness values of ALL individuals in population
    allPopFit = [ind.fitness.values for ind in pop]
    allfalseNeg = [fv[1] for fv in allPopFit]
    allfalsePos = [fv[0] for fv in allPopFit]
    
    # collects fitness values of PARETO individuals in population
    tempHof = tools.ParetoFront()
    tempHof.update(pop)
    fitnessValues = [ind.fitness.values for ind in tempHof]
    fitnessValues.sort(key=lambda t: t[1]) 
    # sorts fitness values in order of x - co-ordinate
    # i.e. false negatives
    falseNeg = [fv[1] for fv in fitnessValues]
    falsePos = [fv[0] for fv in fitnessValues]
    
    auc = 0
    oldX = 0
    # computes auc as a Reimann sum
#     for fv in fitnessValues:
# #         print(fv)
#         auc += (fv[1] - oldX) * fv[0]
#         oldX = fv[1]
    for i in range(len(fitnessValues) - 1):
        height = fitnessValues[i][0]
        width = fitnessValues[i + 1][1] - fitnessValues[i][1]
        auc += height * width
        
    # does actual plotting of the graph as a steps-post
    # will look weird if elements being plotted are not pareto
    plt.scatter(allfalseNeg, allfalsePos, color='g')
    plt.scatter(falseNeg, falsePos, color='C1')
    plt.plot(falseNeg, falsePos, color='C1', drawstyle='steps-post')
    plt.xlabel("False Negs")
    plt.ylabel("False Posits")
    plt.text(180,180, 'AUC: %i' %auc)
    plt.title("Generation: %i" % (g+1))
    fileName = "Generation %i" % (g+1)
    plt.savefig(fileName)
    plt.close()
    # returns computed auc value for the passed in population
    return auc

In [16]:
# Begin the evolution

for g in gen:
    # print("-- Generation %i --" % g)
    
   #  AT THIS POINT
   #  Update the hall of fame with the new population 
    hof.update(pop)
    # The Pareto front hall of fame contains all the non-dominated individuals that ever lived in the population.
    # That means that the Pareto front hall of fame can contain an infinity of different individuals.

    # Select the next generation individuals
    # Calls selTournament method within toolBox
    offspring = toolbox.select(pop, len(pop)) # MIGHT HAVE TO USE fit_attr ARGUMENT
    # offspring now contain relatively fitter individuals. 
    # There is a risk of the best individuals not making it and the weaker ones going through
    # In cases when the stronger individuals never compete in the tournament
    # This is where the hall of fame comes in to preserve elitism
    
    # Clone the selected individuals
    # Creates a deep copy of individuals so we can alter them without affecting originals
    offspring = list(map(toolbox.clone, offspring))
    # the individual in offspring are the ones who have emerged victorious from the selTournament
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CX_PB:
            # Mate children with probability CX_PB
            toolbox.mate(child1, child2)
            # DELETION is done to identify those indiviuals that mated
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUT_PB:
            # Mutate children with probability MUT_PB
            toolbox.mutate(mutant)
            # DELETION is done to identify those indiviudals that were mutated
            del mutant.fitness.values

    # At this point individuals that were mutated or mated have invalid fitnesses
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # THESE LINES NEED TO BE CHECKED
    # BELIEVE THAT SOME BEST INDIVIDUALS ARE BEING LOST

    # Replace population with new, fitter offspring
    offspring = getBestFromPopulation(offspring)
    
    # gets auc of the passed paretoFront
    auc = dispGraphwithAUC(offspring)
    
    # END HERE. REST SHOULD WORK
    
    # adds computed auc to the aucList
    aucList += [auc]
    
    pop[:] = offspring
    print("-- Generation %i --, done" % (g+1))

    # Gather all the fitnesses in one list and print the stats
#     fits = [ind.fitness.values[0] for ind in pop]

#     length = len(pop)
#     mean = sum(fits) / length
#     sum2 = sum(x*x for x in fits)
#     std = abs(sum2 / length - mean**2)**0.5

#     g_max = max(fits)
#     g_min = min(fits)
        
#     avg_list.append(mean)
#     max_list.append(g_max)
#     min_list.append(g_min)

#     print("  Min %s" % g_min)
#     print("  Max %s" % g_max)
#     print("  Avg %s" % mean)
#     print("  Std %s" % std)


print("-- End of (successful) evolution --")

# best_ind = tools.selBest(pop, 1)[0]
# for famer in hof:
#     print dist = famer.fitness.values
# print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

-- Generation 1 --, done
-- Generation 2 --, done
-- Generation 3 --, done
-- Generation 4 --, done
-- Generation 5 --, done
-- Generation 6 --, done
-- Generation 7 --, done
-- Generation 8 --, done


KeyboardInterrupt: 

In [ ]:
# TAKES THE BEST INDIVIDUAL FROM THE POPULATION 
# CHOOSES BASED ON DISTANCE TO ORIGIN

best_famer = None
best_distance = -1

# print('HOF: ', len(hof))
# for famer in hof:
#     print(famer.fitness.values)

# print('pop: ', len(pop))
pop += list(hof)

# print(len(pop))
for ind in pop:
#     print(ind.fitness.values)
    FP, FN = ind.fitness.values
    dist = (FP * FP + FN * FN) ** 0.5
    if best_distance == -1:
        best_distance = dist
        best_famer = ind
    elif dist < best_distance:
        best_distance = dist
        best_famer = ind

print("best individual: ", best_famer.fitness.values)

# prints the list of auc's over generation
print(aucList)
gens = range(1, len(aucList) + 1)
# plots the aucList and displays trend in AUC
plt.scatter(gens, aucList, color='b')
plt.plot(gens, aucList, color='b', drawstyle='default')
plt.xlabel("Generation")
plt.ylabel("AUC")
plt.title("AUC over Generations")
plt.savefig("AUC over Generations")
plt.show()


In [ ]:
None == None
